<a href="https://colab.research.google.com/github/Vaani64/Deep_Learning_Codes/blob/main/Implementation_of_a_Restricted_Boltzmann_Machine_(RBM)_that_demonstrates_stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate synthetic dataset
def generate_data(samples=1000, features=50):
    np.random.seed(42)
    X = np.random.rand(samples, features) > 0.5  # Random binary data
    y = (np.sum(X, axis=1) > (features / 2)).astype(int)  # Binary label: 1 if sum of features > features/2
    return X, y

# Train stacked RBMs
class StackedRBMs:
    def __init__(self, rbm_layers, rbm_learning_rate=0.1, rbm_n_iter=10):
        # Create a list of RBM layers
        self.rbms = [
            BernoulliRBM(n_components=n_components, learning_rate=rbm_learning_rate, n_iter=rbm_n_iter)
            for n_components in rbm_layers
        ]
        self.logistic_regression = LogisticRegression()

    def pretrain(self, X):
        print("Starting unsupervised pretraining with RBMs...")
        current_input = X
        for idx, rbm in enumerate(self.rbms):
            print(f"Training RBM layer {idx + 1}/{len(self.rbms)}...")
            rbm.fit(current_input)  # Train the RBM
            current_input = rbm.transform(current_input)  # Use transformed data as input for next RBM

    def fine_tune(self, X, y):
        print("Starting supervised fine-tuning with Logistic Regression...")
        # Pass data through all RBMs
        for rbm in self.rbms:
            X = rbm.transform(X)
        self.logistic_regression.fit(X, y)  # Train logistic regression on the transformed features

    def predict(self, X):
        # Pass data through all RBMs
        for rbm in self.rbms:
            X = rbm.transform(X)
        return self.logistic_regression.predict(X)  # Use logistic regression for final prediction


# Main execution
if __name__ == "__main__":
    # Generate data
    X, y = generate_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define RBM stacking parameters
    rbm_layers = [128, 64]  # Two RBMs with 128 and 64 hidden units respectively

    # Initialize Stacked RBMs
    stacked_rbm = StackedRBMs(rbm_layers=rbm_layers, rbm_learning_rate=0.05, rbm_n_iter=15)

    # Pretraining (unsupervised)
    stacked_rbm.pretrain(X_train)

    # Fine-tuning (supervised)
    stacked_rbm.fine_tune(X_train, y_train)

    # Evaluate model
    y_pred = stacked_rbm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


Starting unsupervised pretraining with RBMs...
Training RBM layer 1/2...
Training RBM layer 2/2...
Starting supervised fine-tuning with Logistic Regression...
Test Accuracy: 85.00%
